In [1]:
import os
import nibabel as nib
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
import numpy as np
import pandas as pd

In [2]:
# Define the folder path containing NIfTI files
folder_path = "F:\Image CHD\ImageCHD_dataset\Test Data"

In [3]:
# Define output file path
output_file = "glcm_features2.xlsx"

In [4]:
# Define property names for clarity
property_names = ["Contrast", "Dissimilarity", "Homogeneity", "Energy", "Correlation"]

In [5]:
# List to store data for each slice
data_list = []

In [6]:
# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if it's a NIfTI file
    if not filename.endswith(".nii.gz"):
        continue

    # Load the image
    img = nib.load(os.path.join(folder_path, filename))
    data = img.get_fdata()

    # Iterate through each slice
    for slice_num in range(data.shape[2]):
        # Extract a single 2D slice from the 3D volume
        slice_data = data[:, :, slice_num]

        # Convert to uint8
        slice_data_uint8 = (slice_data * 255).astype(np.uint8)

        # Calculate GLCM and properties
        glcm = graycomatrix(
            slice_data_uint8, distances=[1], angles=[0], symmetric=True, normed=True
        )
        properties = [
            graycoprops(glcm, prop)[0, 0] for prop in ["contrast", "dissimilarity", "homogeneity", "energy", "correlation"]
        ]

        # Calculate additional properties
        mean_val = np.mean(slice_data_uint8)
        variance_val = np.var(slice_data_uint8)
        entropy_val = shannon_entropy(slice_data_uint8)
        second_moment_val = np.mean(slice_data_uint8 ** 2)

        # Prepare data for each slice
        slice_data = {
            "File Name": filename,
            "Slice Number": slice_num + 1,
            "Mean": mean_val,
            "Variance": variance_val,
            "Entropy": entropy_val,
            "Second Moment": second_moment_val
        }

        # Add GLCM properties
        for idx, prop_name in enumerate(property_names):
            slice_data[prop_name] = properties[idx]

        # Append data to list
        data_list.append(slice_data)

In [7]:
# Check if data_list is non-empty
if not data_list:
    print("No NIfTI files found in the specified directory.")
    exit()

In [8]:
# Create and save the DataFrame
data_frame = pd.DataFrame(data_list)
data_frame.to_excel(output_file, index=False)

print(f"GLCM features saved to {output_file}")

GLCM features saved to glcm_features2.xlsx
